In [1]:
#иной подход к TEA: теперь адаптер будем ставить не в параллель с LM-head, а в параллель с трансформерным слоем. 
#И адаптер = трансформерные слои, чередующиеся с резнетами

In [2]:
import sys
import time
import json
import os
from collections import OrderedDict
from pathlib import Path
from collections import deque
import os, pickle, random
import psutil
import warnings
import codecs
import numpy as np
import pandas as pd
import requests
from typing import Dict, List, Optional
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModel
from transformers import StoppingCriteria, StoppingCriteriaList
from torch import cuda, LongTensor, FloatTensor
from peft import PeftModel, PeftConfig, PeftModelForCausalLM

import ensembles

In [3]:
device = torch.device("cuda")

In [4]:
from sequential_models import GPTAdapterLayer

In [5]:
start_train = True#Запускаем ли мы сейчас обучение с нуля, или с какого-то чекпоинта



cardinality = 128256#размер словаря токенов

padding_token = 128009
forbidden_tokens_list = [padding_token]
seed = int(np.random.rand() * 1000000)#random seed for data sampling

bits_per_number = 4

In [6]:
#конфигурации моделек
mode = 'finetune'#'finetune', 'pretrain'
model_version = "light_1"
if model_version == "light_1":
    embed_dim = 2048
    model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit"
    num_heads_tlayer = 16
    ff_dim = embed_dim  * 4
    t_layers_count = 2
    layer_configs = [2048, 2048, 2048, 2048]
    batch_size = 2
    accum_batch = 2
    lr = 1e-3
    opt_type = 'adam'
elif model_version == "heavy_1":
    embed_dim = 4096
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
    num_heads_tlayer = 16
    ff_dim = embed_dim  * 2
    t_layers_count = 2
    layer_configs = [2048, 2048, 2048, 2048]
    batch_size = 2
    accum_batch = 10
    lr = 1e-3
    opt_type = 'adam'

In [7]:
path2tadapter = f"tadapter_{model_version}.pth"
path2lm_head = f"lmhead_{model_version}.pth"

In [8]:
tokenizer = AutoTokenizer.from_pretrained("unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit")#, cache_dir="D:\cache\huggingface\\"+ model_name)

if bits_per_number == 4:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4"#, bnb_4bit_compute_dtype=torch.bfloat16
    )
elif bits_per_number == 8:
    bnb_config = BitsAndBytesConfig(
        load_in_8bit=True, bnb_8bit_use_double_quant=True, bnb_8bit_quant_type="nf8"#, bnb_8bit_compute_dtype=torch.bfloat16
    )
else:
    bnb_config = None

tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
              model_name,
              #device_map="auto",
              #device_map=device,
              #torch_dtype=torch.bfloat16,
              quantization_config=bnb_config,
              cache_dir="D:\cache\huggingface\\"+ model_name)


<>:21: SyntaxWarning: invalid escape sequence '\c'
<>:21: SyntaxWarning: invalid escape sequence '\c'
/tmp/ipykernel_2828930/2897674363.py:21: SyntaxWarning: invalid escape sequence '\c'
  cache_dir="D:\cache\huggingface\\"+ model_name)
/home/ssdovgan/main/lib/python3.12/site-packages/transformers/quantizers/auto.py:212: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [9]:
# Исходная моделька


# Замена последнего слоя. Тут трансформер, то можно и твой резнет... 
initial_layer = model.model.layers[-1]

#создать модель
torch.manual_seed(1)
np.random.seed(1)
random.seed(1)

model.model.layers[-1] = GPTAdapterLayer(initial_layer, 
                                         dim=embed_dim, 
                                         num_heads=num_heads_tlayer, 
                                         ff_dim=ff_dim, 
                                         t_layers_count=t_layers_count, 
                                         layer_configs=layer_configs,
                                         conservativity=conservativity ).to(device)

if start_train:
    pass
else:
    model.model.layers[-1].corrector_stack = torch.load(path2tadapter, weights_only=False).to(device)
    #model.lm_head = torch.load(path2lmhead, weights_only=False).to(device)
    model.lm_head.load_state_dict(torch.load(path2lmhead))

# Заморозка всей модели
for param in model.parameters():
    param.requires_grad = False

# Разморозка последнего блока
for param in model.model.layers[-1].corrector_stack.parameters():
    param.requires_grad = True
#И lm-head
model.lm_head.float()
for param in model.lm_head.parameters():
    param.requires_grad = True
model.model.layers[-1].transformer_float_mode = 32
model.model.layers[-1].lmhead_float_mode = 32



def count_trainable_parameters(model: nn.Module) -> tuple[int, int]:
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    frozen = sum(p.numel() for p in model.parameters() if not p.requires_grad)
    return trainable, frozen
trainable, frozen = count_trainable_parameters(model)
print(f"Обучаемые: {trainable:,}\nЗамороженные: {frozen:,}")

use_memnets True
use_memnets True
Обучаемые: 180,251,602
Замороженные: 749,275,136


In [10]:
if opt_type == 'adam':
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
else:
    momentum = 0.
    optimizer = torch.optim.SGD(model.parameters(), lr=lr*0.01, momentum=momentum)

In [11]:
class InstructDatasetR(Dataset):
    def __init__(self, data_file, tokenizer, max_seq_length, cut=None):
        self.data_file = data_file
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length
        self.cut = cut
        self.data = self.load_data()
        self.log_samples = deque(maxlen=45)

    def load_data(self):
        with open(self.data_file, 'rb') as f:
            data = pickle.load(f)
        random.seed(seed)
        data = random.sample(data, len(data))
        # if self.cut is None:
        #     data = [data_cur for data_cur in data if len(data_cur[1])>5]
        # else:
        #     data = [data_cur for data_cur in data[:self.cut] if self.cut>5]
        # for i in range(100):
        #     print('***', data[i])
        #     try:
        #         if (not 'Алис' in data[i][0]) and (not 'Элеон"' in data[i][0]) and (not 'Софи"' in data[i][0]) and (not 'Сэм"' in data[i][0]) (not 'Алис' in data[i][1]) and (not 'Элеон"' in data[i][1]) and (not 'Софи"' in data[i][1]) and (not 'Сэм"' in data[i][1]):
        #             print('***', data[i])
        #     except Exception:
        #         pass
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        self.log_samples.append(self.data[idx])
        parts = self.data[idx]
        text = parts[0]
        label = parts[1]
        parts = [parts[0], parts[1], 1]
        for r_variant in [-2,-1,-0.5,0.5,1,2]:
            s = f"<r{r_variant}>"
            if s in label:
                parts[-1] = r_variant
                label = label.replace(s, '')
                break
        r = parts[-1]
        if text is None:
            text = label
        
        # Кодируем текст и метку с помощью tokenizer
        encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.max_seq_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids']
        attention_mask = encoding['attention_mask']
        
        # Кодируем метку
        label_encoding = self.tokenizer.encode_plus(
            label,
            max_length=self.max_seq_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        label_ids = label_encoding['input_ids']
        return {
            'input_ids': input_ids[0],
            'attention_mask': attention_mask,
            'labels': label_ids[0],
            'mult': torch.tensor(r)
        }

In [12]:
#настройки датасета

if mode == 'pretrain':
    #претрейн
    max_seq_len = 6900
    max_seq_len4inference = 6850
    # max_seq_len = 4500
    # max_seq_len4inference = 4450
    # max_seq_len = 3800
    # max_seq_len4inference = 3750
    # max_seq_len = 3600
    # max_seq_len4inference = 3550
else:
    #дообучение
    # max_seq_len = 1950
    # max_seq_len4inference = 1900
    max_seq_len = 2050
    max_seq_len4inference = 2000

print('batch_size', batch_size)

batch_size 2


In [13]:
# Создание Dataset
if mode == 'pretrain':
    #dataset = InstructDatasetR("./data/dataset_llm_full_mflat.pkl", tokenizer, max_seq_len, cut=None)
    dataset = InstructDatasetR("../data_tea/dataset_llm_full_mflat.pkl", tokenizer, max_seq_len, cut=None)
else:
    dataset = InstructDatasetR("../data_tea/dataset_llm_full_instruct.pkl", tokenizer, max_seq_len, cut=None)
len(dataset)

663531

In [14]:
def soft_clip(tens, mn, mx):
    mn = torch.tensor(mn).to(tens.device)
    mx = torch.tensor(mx).to(tens.device)
    tens[tens<mn] = torch.nn.Sigmoid()(tens[tens<mn]) + mn
    tens[tens>mx] = torch.nn.Sigmoid()(tens[tens>mx]) + mx
    return tens
# Функция для подсчёта количества последних токенов, равных padding_token
def count_padding(tensor, padding_token):
    counts = []
    for row in tensor:
        count = 0
        for token in reversed(row):
            if token.item() == padding_token:
                count += 1
            else:
                break
        counts.append(count)
    return min(counts)

In [15]:
loss_fct = nn.CrossEntropyLoss(reduction = 'none')

In [16]:
proba_drop_idx = 0.04

In [21]:
#проинференсить модель на батчах, сделать датасет для постпроцессинга
print('batch_size', batch_size)
i_pointer = 0

batch_accum_counter = 0
loss_array = []
metr_array = []
while 1: 
    input_ids_full = []
    labels_full = []
    weights_full = []
    #for i in range(batch_size):
    i = 0
    offset_counter = 0
    while i < batch_size:
        offset_counter += 1
        sample_cur  = dataset[i_pointer + i]
        input_ids = sample_cur['input_ids']
        labels = sample_cur['labels']
        weights = sample_cur['mult']
        
        input_ids_len = len(input_ids)
        labels_len = len(labels)
        input_ids = input_ids[input_ids!=padding_token]
        
        drop_idx = torch.rand_like(input_ids.to(torch.float)) < proba_drop_idx
        input_ids = input_ids[~drop_idx]
        
        labels = labels[labels!=padding_token]
        padding_size = input_ids_len + labels_len - len(input_ids) - len(labels)
        input_ids_cur = torch.cat([input_ids, labels, torch.zeros(padding_size, dtype=torch.int32) + padding_token])[:max_seq_len4inference]
        labels_cur = torch.cat([input_ids * 0 + padding_token, labels, torch.zeros(padding_size, dtype=torch.int32) + padding_token])[:max_seq_len4inference]
        #print('fact_size', np.min([len(input_ids) + len(labels), max_seq_len4inference]), len(input_ids) + len(labels))
        if torch.all(labels_cur) == padding_token:
            print('empty label')
            continue
        i += 1
        
        input_ids_full.append(input_ids_cur[:-1])
        labels_full.append(labels_cur[1:])
        weights_full.append(weights)

        # if not torch.any(input_ids!=0):
        #     #X + Y
        #     print(list(input_ids_cur.numpy()))
        #     print(list(labels_cur.numpy()))
        #     1/0
    i_pointer += i #should be batch_size
    #print('A', pd.Timestamp.now())
    
    input_ids_full = torch.stack(input_ids_full).to('cuda')
    labels_full = torch.stack(labels_full).to('cuda')
    weights_full = torch.stack(weights_full).to('cuda').T

    cnt_pads = count_padding(labels_full, padding_token)
    if cnt_pads >= input_ids_full.shape[1] - 5:
        continue
    if cnt_pads > 0:
        input_ids_full = input_ids_full[:, :-cnt_pads]
        labels_full = labels_full[:, :-cnt_pads]
            
        outp = model.forward(input_ids_full)
        logits = outp['logits']
        del outp
        #outp['states'] = outp['hidden_states'][-1]
    # {"loss": loss_agg, "logits": logits, "states":embeddings}
        #outp['states'] = outp['states'][:, :input_ids_full.shape[1]]
        #state_cur = outp['states'].reshape([outp['states'].shape[0] * outp['states'].shape[1], outp['states'].shape[2]])#.cpu().to(torch.float16).numpy()
        weights_full2d = torch.vstack([weights_full] * labels_full.shape[-1]).to('cuda').T
        labels_cur = labels_full.ravel()#.cpu().numpy()
        weights_cur = weights_full2d.ravel()

        del labels_full
        del weights_full2d
        del weights_full
        idx = torch.isin(labels_cur, torch.tensor(forbidden_tokens_list).to(device))
        labels_cur = labels_cur[~idx]
        #state_cur = state_cur[~idx]
        weights_cur = weights_cur[~idx]
    else:
        continue
    print(1)
    if labels_cur.shape[0] == 0:
        continue

    ce_loss = loss_fct(logits.view([logits.shape[0] * logits.shape[1], logits.shape[2]])[~idx], labels_cur.view(-1))
    print(2)
    ce_loss = ce_loss * weights_cur.to('cuda')
    ce_loss = soft_clip(ce_loss, -0.6, 10.)
    loss = torch.mean(ce_loss)
    loss.backward()
    
    #print('D', pd.Timestamp.now())

    acc = torch.mean((torch.argmax(logits.view([logits.shape[0] * logits.shape[1], logits.shape[2]])[~idx], axis=-1) == labels_cur).to(torch.float16))
    del logits
    del labels_cur
    batch_accum_counter += 1
    loss_array += [torch.mean(ce_loss).item()]
    metr_array += [acc.item()]
    if batch_accum_counter >= accum_batch:
        optimizer.step()
        optimizer.zero_grad()
        torch.cuda.empty_cache()
        print('ce_loss', np.round(np.mean(loss_array), 3), i_pointer, 'from', len(dataset), np.round(100 * i_pointer/len(dataset), 5), '%', 'acc', np.mean(metr_array), pd.Timestamp.now())
        loss_array = []
        metr_array = []
        batch_accum_counter = 0

        
        
        if (np.random.rand() < 1 and accum_batch > 100) or (np.random.rand() < 0.1 and accum_batch <= 100):
            model.lm_head.training = False
            torch.save(model.model.layers[-1].corrector_stack, path2tadapter)
            torch.save(model.lm_head, path2lmhead)

            if np.random.rand()<0.2:
                torch.save(model.model.layers[-1].corrector_stack, path2tadapter + '.back')
                torch.save(model.lm_head, path2lmhead + '.back')
            model.lm_head.training = True

batch_size 2
1
2
1
2
ce_loss 3.164 6 from 663531 0.0009 % acc 0.465576171875 2025-04-11 00:31:08.054529
1
2
1
2
ce_loss 2.489 10 from 663531 0.00151 % acc 0.544677734375 2025-04-11 00:31:09.286413
1
2
1
2
ce_loss 4.113 16 from 663531 0.00241 % acc 0.333251953125 2025-04-11 00:31:10.484526
1
2


KeyboardInterrupt: 

In [ ]:
#если принудительно стопнули, можно добить батч
optimizer.step()
optimizer.zero_grad()
torch.cuda.empty_cache()
print('ce_loss', np.round(np.mean(loss_array), 3), i_pointer, 'from', len(dataset), np.round(100 * i_pointer/len(dataset), 5), '%', 'acc', np.mean(metr_array), pd.Timestamp.now())

In [ ]:
#Засейвить
model.lm_head.training = False
torch.save(model.model.layers[-1].corrector_stack, path2tadapter)
torch.save(model.lm_head, path2lmhead)
model.lm_head.training = True

In [ ]:
#check generation

model.model.layers[-1].half()
model.lm_head.half()
model.model.layers[-1].transformer_float_mode = 16
model.model.layers[-1].lmhead_float_mode = 16


prompt = "Превед медвед"
inputs = tokenizer(prompt, return_tensors="pt").to(device)
t = pd.Timestamp.now()
generate_ids = model.generate(inputs.input_ids.to(device),
                                        #attention_mask=inputs["attention_mask"],
                                        max_new_tokens=200,
                                        pad_token_id=tokenizer.pad_token_id,
                                        eos_token_id=tokenizer.eos_token_id,
                                        bos_token_id=tokenizer.bos_token_id,
                                        #no_repeat_ngram_size=3,
                                        #penalty_alpha=penalty_alpha,
                                        early_stopping=True,
                                        use_cache=True,
                                        num_beams=1,
                                        tokenizer=tokenizer)
print(pd.Timestamp.now() - t)
answer = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(answer)